In [1]:
import pandas as pd

trainingSet = pd.read_csv("./data/train.csv")
testingSet = pd.read_csv("./data/test.csv")

In [38]:
trainingSet[:200]

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,0,0005019281,ADZPIG9QOCDG5,0,0,4.0,1203984000,good version of a classic,This is a charming version of the classic Dick...
1,1,0005019281,A35947ZP82G7JH,0,0,3.0,1388361600,Good but not as moving,It was good but not as emotionally moving as t...
2,2,0005019281,A3UORV8A9D5L2E,0,0,3.0,1388361600,Winkler's Performance was ok at best!,"Don't get me wrong, Winkler is a wonderful cha..."
3,3,0005019281,A1VKW06X1O2X7V,0,0,5.0,1202860800,It's an enjoyable twist on the classic story,Henry Winkler is very good in this twist on th...
4,4,0005019281,A3R27T4HADWFFJ,0,0,4.0,1387670400,Best Scrooge yet,This is one of the best Scrooge movies out. H...
...,...,...,...,...,...,...,...,...,...
195,195,0005119367,A3AW8XP45YVIIZ,5,5,5.0,1172620800,Superb dramatization of the Old Testament stor...,This film is a recounting of the Genesis tale ...
196,196,0005119367,A2YN8XPTHXC41Q,1,1,5.0,1376524800,Great DVD,This was a great DVD. It tells the story of Jo...
197,197,0005119367,A1LSHLA6SY0N1S,0,0,NaN,1319932800,Unlimited by Time,How wonderful that a story about precognitive ...
198,198,0005119367,ASJMQXL151RHN,1,1,5.0,1384300800,Joseph....A Godly Man,Joseph clearly tells how God can use anyone. ...


In [2]:
predictionSet = pd.merge(trainingSet, testingSet, left_on='Id', right_on='Id')
print(predictionSet.columns)

predictionSet = predictionSet.drop(columns=['Score_x'])
predictionSet = predictionSet.rename(columns={'Score_y': 'Score'})

print(predictionSet.columns)
# predictionSet.to_csv("./data/prediction.csv", index=False)

X_train = trainingSet[trainingSet['Score'].notnull()]

Index(['Id', 'ProductId', 'UserId', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score_x', 'Time', 'Summary', 'Text',
       'Score_y'],
      dtype='object')
Index(['Id', 'ProductId', 'UserId', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Time', 'Summary', 'Text', 'Score'],
      dtype='object')


In [28]:
df = X_train.drop(columns=['HelpfulnessNumerator', 'HelpfulnessDenominator', 'ProductId', 'UserId', 'Time', 'Summary'])
#X_simple = pd.DataFrame({"Text": X_simple["Text"], "Score" : X_simple["Score"]})
df=df.dropna(axis=0)
print(df.isnull().sum())
df[:10]

Id       0
Score    0
Text     0
dtype: int64


,Id,Score,Text
0,0,4.0,This is a charming version of the classic Dick...
1,1,3.0,It was good but not as emotionally moving as t...
2,2,3.0,"Don't get me wrong, Winkler is a wonderful cha..."
3,3,5.0,Henry Winkler is very good in this twist on th...
4,4,4.0,This is one of the best Scrooge movies out. H...
6,6,5.0,This is the American adaptation of the Charles...
7,7,5.0,Glad that this american classic came out on dv...
8,8,5.0,A good Christmas carol dhenry winkler one duri...
9,9,5.0,How a bitter old man comes to know the true me...
10,10,5.0,"The small historic Canadian town of Elora, wit..."


In [18]:
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [29]:
vec = TfidfVectorizer(stop_words="english")
vec.fit(df.Text.values)
features = vec.transform(df.Text.values)

In [31]:
cls = MiniBatchKMeans(n_clusters=5, random_state=0)
cls.fit(features)

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=5, n_init=3, random_state=0, reassignment_ratio=0.01,
                tol=0.0, verbose=0)

In [34]:
from sklearn.metrics import homogeneity_score
homogeneity_score(df.Score, cls.predict(features))

0.003846166305595842